# Пример построения интерактивного графика с использованием библиотеки plotly.express и jupyter_dash

Материалы курсов "Исследования скважин и пластов" и "Гидродинамические исследования скважин" в РГУ нефти и газа имени И.М.Губкина.


версия 0.3 от 20.10.2023

Для построения интерактивных графиков и простых веб приложений может оказаться удобным использовать библиотеку `dash`, которая позволяет делать все прямо в тетрадке jupyter или в колабе

Для установки библиотеки надо вызвать в терминале команду
`pip install dash`

В колабе это можно сделать выполнив команду 
`!pip install dash` в ячейке тетрадки

In [2]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

import pandas as pd
import numpy as np
import scipy.special as sc

In [3]:
# Решение линейного стока уравнения фильтрации
def pd_ei(rd, td):
  return -1/2*sc.expi(-rd**2 / 4 / td)

In [5]:
# готовим датафрейм который будет отображаться
df = pd.DataFrame(columns = ['pd', 'td', 'rd'])
# готовим массив значений расстояний для которого будут строиться графики
df['rd'] = np.logspace(0.1, 4, 100)

# создаем интерактивное приложение plotly dash 
app = Dash(__name__)
# описываем внешний вид приложения на основе html разметки
app.layout = html.Div([
    html.H1("JupyterDash Demo test 2"), # заголовок
    dcc.Graph(id='graph'), # зона отображения графика
    html.Label(["td"]), # надпись под графиком
    dcc.Slider(
            id='my-slider',
            min=2,
            max=8,
            step=0.1,
            value=4,
            ),    # слайдер для смены значения времени
    dcc.Checklist(
        id='my-check',
        options=[
            {'label': 'semi log', 'value': 'semilog'}
        ],
            value=['semi log', 'semilog']
        )  # переключатель вида оси х
])

# определим функции отрисовки графика - callback функция
# она автоматически будет вызываться dash каждый раз когда надо будет нарисовать график
# в декораторе указывает откуда будут браться аргументы функции отрисовки
@app.callback(
    Output('graph', 'figure'),
    Input("my-slider", "value"),
    Input("my-check", "value"),    
)
def update_figure(td, log):
    # функция отрисовки 
    df['td'] = round(10**td)
    df['pd'] = pd_ei(df['rd'], df['td'])
    return px.line(
        df, x="rd", y="pd", color = "td",
        title="time = "+str(round(10**td)),
        log_x = log==['semi log', 'semilog']
    )
# Run app and display result inline in the notebook
app.run(mode='inline')